### First we have to determine the AZOV domain inside GLORYS corse grid, for that we have to find the indexes with which we will cut GLORYS
### We will find these indexes by finding the common grid point between GLORYS025 and AZOV12

In [34]:
## imports

import numpy as np
import xarray as xr
from netCDF4 import Dataset

%matplotlib inline

In [35]:
## datasets

#AZOV12
coord_azov="/media/extra/DATA/coordinates_AZOV12.nc"

#GLORYS

glorys="/media/extra/DATA/GLORYS/GLORYS2V4_ORCA025_201001_gridT.nc"


In [36]:
dscoAZ=xr.open_dataset(coord_azov,decode_times=False)
dsglo=xr.open_dataset(glorys)

latAZ=dscoAZ.gphit
lonAZ=dscoAZ.glamt

latGLO=dsglo.nav_lat
lonGLO=dsglo.nav_lon


In [37]:
latAZ.shape

(1, 1, 223, 215)

### We have to find inside GLORYS grid the common point with AZOV grid. Since AZOV's resolution is 3x higher than GLORYS this common point must in the first 3x3 square of AZOV

In [38]:
latAZcoin=latAZ[0,0,0:3,0:3]
print latAZcoin

<xarray.DataArray 'gphit' (y: 3, x: 3)>
array([[ 39.943066,  39.939525,  39.935982],
       [ 39.996226,  39.992655,  39.989081],
       [ 40.049299,  40.045697,  40.042094]])
Coordinates:
    time     float32 0.0
Dimensions without coordinates: y, x


In [39]:
lonAZcoin=lonAZ[0,0,0:3,0:3]
print lonAZcoin

<xarray.DataArray 'glamt' (y: 3, x: 3)>
array([[ 25.651459,  25.734657,  25.817852],
       [ 25.655329,  25.738524,  25.821717],
       [ 25.659231,  25.742424,  25.825614]])
Coordinates:
    time     float32 0.0
Dimensions without coordinates: y, x


### For these 9 grid points we are looking in GLORYS if there is an exact match

In [40]:
for i in np.arange(0,3):
    for j in np.arange(0,3):
        print i,j,lonAZcoin[i,j].values,latAZcoin[i,j].values
        indBOX=np.where((lonGLO==lonAZcoin[i,j])&(latGLO==latAZcoin[i,j]))
        print indBOX[0], indBOX[1]

0 0 25.6514591641 39.9430656904
[] []
0 1 25.7346565341 39.9395245682
[] []
0 2 25.8178520203 39.9359817505
[679] [1251]
1 0 25.6553290524 39.9962264737
[] []
1 1 25.7385243423 39.9926546344
[] []
1 2 25.821717274 39.9890812532
[] []
2 0 25.659230749 40.0492990834
[] []
2 1 25.742423717 40.0456968714
[] []
2 2 25.8256139873 40.0420939128
[] []


### The only time the index are found is for i=0 and j=2, let's check out

In [41]:
print lonAZcoin[0,2].values,latAZcoin[0,2].values
print lonGLO[679,1251].values,latGLO[679,1251].values


25.8178520203 39.9359817505
25.8178520203 39.9359817505


### So 679,1251 is the bottom left corner of GLORYS that matches the bottom left corner of AZOV
### To get the coordinates of the top right corner we must consider the size of AZOV domain which is 215x223 so for GLORYS which is 3x coarser. So we must have a domain that is 72x75
### So now you can extract AZOV025 from GLORYS with the command ncks -F -d x,1251,1323 -d y,680,754 (in python x and y are reversed and start at 0, and since the corresponding point in AZOV12 is the second we have to take one more on x)

### The domain will be a bit larger but it is okay if you build the weights accordingly

### It will also be useful to know these coordinates in case you want to build some bdy, you will have first to duplicate the bathymetry from GLORYS to AZOV12 at the boundaries considered (here bottom & left) in order to have continuity

### Now that you have your own AZYS025 domain we can construct your own pseudo coordinates based on the size of this domain and the fact that AZOV12 is 3x higher in resolution

### I'm going to use the structure of the coordinates files and replace the real latitude and longitude by false regular latitudes and longitudes

In [42]:
!cp /media/extra/DATA/coordinates_AZOV12.nc /media/extra/DATA/pseudo_coordinates_AZOV12.nc
!ncks -F -d x,1251,1323 -d y,680,754 -v nav_lat,nav_lon /media/extra/DATA/GLORYS/GLORYS2V4_ORCA025_201001_gridT.nc /media/extra/DATA/GLORYS/pseudo-coordinates-AZYS025.nc
